In [1]:
from aocd import get_data
import numpy as np
from scipy.spatial import distance_matrix
from collections import Counter
raw_data = get_data(day=8, year=2025)
data = [[int(item) for item in line.split(",")] for line in raw_data.splitlines()]

## Part 1

In [ ]:
matrix = distance_matrix(data, data, p=2)

# Get indices for upper triangle of matrix (excluding diagonal where i==j)
# This avoids counting each pair twice (since distance i→j == j→i)
# k=1 excludes the diagonal (distance from a box to itself = 0)
i_indices, j_indices = np.triu_indices(len(data), k=1)

# Extract the distances for all unique pairs
distances = matrix[i_indices, j_indices]

# Get indices that would sort distances from smallest to largest
# argsort returns positions, not values — so we can reorder multiple arrays together
order = np.argsort(distances)
i_ordered, j_ordered = i_indices[order][:1000], j_indices[order][:1000]
parent=list(range(len(data)))

# UNION-FIND DATA STRUCTURE
# Initialize: each junction box starts in its own circuit
# parent[i] = i means box i is the "root" (leader) of its own circuit
def find(parent, x):
    """
    Find the root of the circuit containing box x.
    Follow parent pointers until we find a box that points to itself.
    """
    while parent[x] != x:
        x = parent[x]
    return x

for i, j in zip(i_ordered, j_ordered):
    # Find the root of each box's circuit
    pi, pj = find(parent, i), find(parent, j)
    # Only merge if they're in DIFFERENT circuits
    # If pi == pj, they're already connected — nothing to do
    if pi != pj:
        # UNION: merge circuits by making one root point to the other
        parent[pj] = pi

# Find the root for every box (some may have chains of parents)
roots = [find(parent, i) for i in range(len(parent))]
circuits = Counter(roots)

prod = 1
for circuit in circuits.most_common(3):
    prod *= circuit[1]
print(prod)

79056


## Part 2

In [ ]:
i_ordered, j_ordered = i_indices[order], j_indices[order]
parent=list(range(len(data)))

def find_root(parent, x):
    """
    Find root with PATH COMPRESSION optimization.
    After finding the root, update parent[orig] to point directly to root.
    This flattens the tree, making future lookups O(1) instead of O(depth).
    """
    orig = x
    while parent[x] != x:
        x = parent[x]
    parent[orig] = x  # Path compression: shortcut for next time
    return x

for i, j in zip(i_ordered, j_ordered):
    pi, pj = find_root(parent, i), find_root(parent, j)
    if pi != pj:
        parent[pj] = pi
        # Track the last pair that actually merged two circuits
        # This will be the connection that creates the single final circuit
        last_pair = (i, j)

print(data[last_pair[0]][0] * data[last_pair[1]][0])

4639477


Key concepts:  
Distance matrix — efficiently compute all pairwise distances at once  
Upper triangle indices — avoid redundant pairs (i,j) and (j,i)  
argsort — sort one array while keeping track of original positions  
Union-Find — efficient data structure for tracking connected components  
Path compression — optimization that flattens parent chains for faster lookups  